## Import INDUSTR XML to Dataframe

In [1]:
!pip install lxml
import pandas as pd
import numpy as np



In [2]:
#import filenames of directory documents/pmg to pandas dataframe

import os
path = '../industr/'
# path = '/Users/mweber/Dropbox/50_dev_projekte/DVV/dvv_content/industr'
files = os.listdir(path)

print(f"Inserted {len(files)} files")


Inserted 31 files


In [3]:
from datetime import datetime
from lxml import etree

data_collection = []

for file in files:

    # if collection.find_one({"dateiname": file}):
    #     continue

    tree = etree.parse(path + f'{file}')
    root = tree.getroot()

    data = []
    for elem in root.findall('asset')[:5]:
        # print element attribute "magazine"
        print(f"id: {elem.get('id')} {elem.get('magazine')}")
        print(elem.find('article/content/title').text)
    break

id: 2714283 
Praktische Lösungsansätze für signifikante Effizienzsteigerungen 
id: 2717086 
Albtraum
Produktionsstillstand
id: 2714056 
Die Gefahren der
Energiewende 
id: 2715265 
Autonome Mobile Roboter rocken die Lagerlogistik 
id: 2715121 Industrial_Solutions
Weniger Daten,  mehr Output


In [4]:
#---------------------------------------------
# 21.06.2024 added fields quelle_id_embeddings, text_length
#---------------------------------------------

from datetime import datetime

data_collection = []

for file in files:

    # if collection.find_one({"dateiname": file}):
    #     continue

    tree = etree.parse(path + f'{file}')
    root = tree.getroot()

    data = []
    for elem in root.findall('asset'):

        # Metadaten auslesen
        artikel_id = elem.get('id')
        quelle_id = "pi_" + elem.get('magazine')
        jahrgang = ""
        nummer = ""
        datum = elem.get('modified_date')
        seite_start = ""
        seite_ende = ""

        # Inhalt auslesen
        title = elem.find('article/content/title').text
        if elem.find('article/content/subtitle') is not None:
            untertitel = elem.find('article/content/subtitle').text
        else:
            untertitel = "leer"
        text_struktur = elem.find('article/content/text')
        text_content = "".join(text_struktur.itertext())
        text_length = len(text_content)

        # Felder hinzufügen
        ki_abstract = ""
        date = datetime.strptime(datum, "%Y-%m-%dT%H:%M:%SZ")
        text_embeddings = {}
        ki_embeddings = {}
        quelle_id_embeddings = {}
        
        data.append([file, artikel_id, quelle_id, jahrgang, nummer, datum, seite_start, seite_ende, title, untertitel, text_content, ki_abstract, date, text_embeddings, ki_embeddings, quelle_id_embeddings, text_length])
    
    data_collection.extend(data)

# read xml file and convert it to a pandas dataframe
df = pd.DataFrame(data_collection, columns=['dateiname', 'artikel_id', 'quelle_id', 'jahrgang', 'nummer', 'datum', 'seite_start', 'seite_ende', 'titel', 'untertitel', 'text', 'ki_abstract', 'date', 'text_embeddings', 'ki_embeddings', 'quelle_id_embeddings', 'text_length'])
# df.head(20)
print(f"Inserted {len(df)} records into DataFrame")

# print list of quelle_id grouped by count sorted by quelle_id
print(df.groupby('quelle_id').size().sort_values(ascending=False))


Inserted 10612 records into DataFrame
quelle_id
pi_AuD                     2555
pi_PuA                     2080
pi_EuE                     1857
pi_E20                     1434
pi_Industry_Forward         869
pi_Industrial_Solutions     852
pi_Next_Technology          813
pi_                         152
dtype: int64


## MongoDB: Import Articles

In [5]:
import os
from dotenv import load_dotenv

from pymongo import MongoClient
from bson import ObjectId

# Init MongoDB Client
load_dotenv()
mongoClient = MongoClient(os.environ.get('MONGO_URI_DVV'))
database = mongoClient.dvv_content_pool
collection = database.dvv_artikel

In [6]:
# Insert data into MongoDB usinf insert_many
data_input = df.to_dict(orient='records')
collection.insert_many(data_input)

print(f"Inserted {len(data_input)} records into MongoDB")

mongoClient.close()

Inserted 10612 records into MongoDB


## Add embeddings

In [7]:
import chatdvv_module as myapi

/opt/homebrew/Caskroom/miniconda/base/envs/chatdvv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
myapi.generate_embeddings(input_field="quelle_id", output_field="quelle_id_embeddings", max_iterations=50000)



Generated embeddings for 10612 records.
